# Tabular Data

## DataFrames

### Exercise 7
Use the same data as exercise 6, but using a fresh notebook. Perform the following:  

a. Download a table (cross-sectional) containing reference data for each ticker. Reference data should include the full company name and any other useful column data you can
find.  
b. Create a ‘transactions’ table, which has three columns: Date, Ticker, Amount, and BuySell. Add a bunch of transactions (at least ten) of your favorite stocks – both buys and sells (all long positions, no shorts, no day trading). Transactions should span a period of two years. Keep them self-consistent but somewhat arbitrary.  
c. Create a function that takes a date and returns a (cross-sectional) table containing a snapshot of your portfolio at that point in time (Ticker, Position).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; i. The function should join the table with your stock reference data, to enrich with the full company name, etc.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ii. The function should join the table with the stock data table, to retrieve the open price, volume, and return for the given date, for each ticker.  
d. Create a ‘scaffold’ table, which is the concatenation of the above function, for every business date over the two-year period. Be sure to add a ‘Date’ column.
e. ‘PnL’ (Profit and Loss) is your total portfolio profit/loss at any given point in time. For a given stock, the daily marked-to-market PnL is simply the return*position. Add a
column containing the daily PnL for each ticker.  
f. Add a column containing the running cumulative PnL per ticker.  
g. Add a column containing the running cumulative PnL for your entire portfolio.  
h. Create a summary table of your average daily PnL, total PnL, and overall yield per ticker.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; i. ‘Yield’ is PnL/Position.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ii. You should exclude days when you had no position in that stock, from the average.  
i. Display your total PnL at the end of the two years.  

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

#### a. Download a table (cross-sectional) containing reference data for each ticker. Reference data should include the full company name and any other useful column data you can find.

In [88]:
# Reload data from 8.2.6
%store -r dataReference
dataReference = dataReference
dataReference = dataReference.rename(columns = {"Symbol":"Ticker"})
dataReference.head()

%store -r dataDaily
hist = dataDaily
hist.head()

,Ticker,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


,Date,Ticker,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,1962-01-02,BA,0.187037,0.0,0.190310,0.187037,0.190310,0.0,352200.0
1,1962-01-02,CAT,0.134703,0.0,0.136015,0.133391,0.134703,0.0,163200.0
2,1962-01-02,CVX,0.050512,0.0,0.050512,0.049714,0.000000,0.0,105600.0
3,1962-01-02,DIS,0.058360,0.0,0.060318,0.058360,0.058360,0.0,817400.0
4,1962-01-02,DTE,0.484267,0.0,0.489250,0.484267,0.000000,0.0,1600.0


#### b. Create a ‘transactions’ table, which has three columns: Date, Ticker, Amount, and BuySell. Add a bunch of transactions (at least ten) of your favorite stocks – both buys and sells (all long positions, no shorts, no day trading). Transactions should span a period of two years. Keep them self-consistent but somewhat arbitrary.

In [89]:
import pandas as pd
import datetime

# Adding transaction to the ledger
# param: 
#  date: str (date of the transaction)
#  ticker: str (stock ticker)
#  amount: int (amount of shares)
#  buysell: str ('Buy' or 'Sell')
# return:
#  pd.Series (raise exception if buysell is incorrect)
def addTransaction(date, ticker, amount, buysell):
    if not (buysell == 'Buy' or buysell == 'Sell'):
        raise Exception('Invalid buysell parameter, only "Buy" or "Sell"')
    else:
        return pd.Series({'Date': datetime.datetime.strptime(date, '%Y-%m-%d'), 
                          'Ticker': ticker, 
                          'Amount': amount, 
                          'BuySell': buysell
                         }
                        )

########################
# Generate transaction header
transactionsHeader = ['Date', 'Ticker', 'Amount', 'BuySell']

# Init the empty df 
ledger = pd.DataFrame (transactions, columns = transactionsHeader)

# Adding transactions to the ledger
ledger = ledger.append(addTransaction('2017-10-21', 'MMM', 15000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2017-10-27', 'ABT', 20000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2017-11-04', 'MMM', 20000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2017-11-11', 'YUM', 22000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2017-12-21', 'MMM', 3000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2018-01-19', 'AMZN', 7000, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2018-02-11', 'AMZN', 1000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2018-06-13', 'AAPL', 9000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2018-08-12', 'ZTS', 7000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2018-10-02', 'YUM', 10000, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-01-07', 'ACN', 20000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-02-20', 'SCHW', 20000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-03-30', 'YUM', 5000, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-04-01', 'YUM', 7000, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-05-09', 'SCHW', 10000, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-06-14', 'SCHW', 10000, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-07-22', 'MMM', 2000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-11-30', 'AAPL', 8000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-11-30', 'MMM', 2000, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2019-12-11', 'ZTS', 2000, 'Sell'), ignore_index=True)

ledger

,Date,Ticker,Amount,BuySell
0,2017-10-21,MMM,15000.0,Buy
1,2017-10-27,ABT,20000.0,Buy
2,2017-11-04,MMM,20000.0,Buy
3,2017-11-11,YUM,22000.0,Buy
4,2017-12-21,MMM,3000.0,Buy
5,2018-01-19,AMZN,7000.0,Sell
6,2018-02-11,AMZN,1000.0,Buy
7,2018-06-13,AAPL,9000.0,Buy
8,2018-08-12,ZTS,7000.0,Buy
9,2018-10-02,YUM,10000.0,Sell


#### c. Create a function that takes a date and returns a (cross-sectional) table containing a snapshot of your portfolio at that point in time (Ticker, Position).

##### i. The function should join the table with your stock reference data, to enrich with the full company name, etc.

In [90]:
x = ledger.join(dataReference.set_index('Ticker'), how='inner', on='Ticker')
x

,Date,Ticker,Amount,BuySell,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,2017-10-21,MMM,15000.0,Buy,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
2,2017-11-04,MMM,20000.0,Buy,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
4,2017-12-21,MMM,3000.0,Buy,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
16,2019-07-22,MMM,2000.0,Buy,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
18,2019-11-30,MMM,2000.0,Buy,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,2017-10-27,ABT,20000.0,Buy,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,2017-11-11,YUM,22000.0,Buy,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
9,2018-10-02,YUM,10000.0,Sell,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
12,2019-03-30,YUM,5000.0,Sell,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
13,2019-04-01,YUM,7000.0,Sell,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997


##### ii. The function should join the table with the stock data table, to retrieve the open price, volume, and return for the given date, for each ticker.

In [91]:
y = ledger.join(hist.set_index(['Date', 'Ticker']), how='inner', on=['Date', 'Ticker'])
y

,Date,Ticker,Amount,BuySell,Close,Dividends,High,Low,Open,Stock Splits,Volume
1,2017-10-27,ABT,20000.0,Buy,52.656242,0.0,52.893990,52.475556,52.770364,0.0,4724300.0
4,2017-12-21,MMM,3000.0,Buy,215.160278,0.0,217.423870,214.821202,216.278328,0.0,2241600.0
5,2018-01-19,AMZN,7000.0,Sell,1294.579956,0.0,1313.000000,1292.989990,1312.000000,0.0,4578500.0
7,2018-06-13,AAPL,9000.0,Buy,46.322815,0.0,46.852359,46.259660,46.740619,0.0,86553600.0
9,2018-10-02,YUM,10000.0,Sell,86.860825,0.0,87.633088,86.744982,87.401410,0.0,1504500.0
10,2019-01-07,ACN,20000.0,Buy,137.587784,0.0,138.375053,135.925775,136.878272,0.0,2315600.0
11,2019-02-20,SCHW,20000.0,Buy,45.228600,0.0,45.296759,44.498321,44.848855,0.0,8832300.0
13,2019-04-01,YUM,7000.0,Sell,97.934021,0.0,98.109286,97.194012,97.787966,0.0,1195200.0
14,2019-05-09,SCHW,10000.0,Sell,44.615162,0.0,44.654111,43.320136,43.719355,0.0,6288800.0
15,2019-06-14,SCHW,10000.0,Sell,40.087196,0.0,41.504569,39.705973,41.377497,0.0,14013400.0


#### d. Create a ‘scaffold’ table, which is the concatenation of the above function, for every business date over the two-year period. Be sure to add a ‘Date’ column.